In [ ]:
import os
import torch
import torchvision
from pytorch_lightning import seed_everything
from pl_bolts.datamodules import CIFAR10DataModule, TinyCIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

PATH_DATASETS = os.environ.get('PATH_DATASETS', '../datasets')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 512 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count()/2)
TEST_ONLY = False

test_transforms = [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),        
        torchvision.transforms.Lambda(lambda x: x.clamp(min = -1, max = 1)),
        torchvision.transforms.Lambda(lambda x: x * 127),   
        torchvision.transforms.Lambda(lambda x: x.floor())  
]

train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
    ] 
    + test_transforms
)

test_transforms = torchvision.transforms.Compose(test_transforms)

cifar10_dm = CIFAR10DataModule(
    data_dir = PATH_DATASETS,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS,
    train_transforms = train_transforms,
    test_transforms = test_transforms,
    val_transforms = test_transforms,
)

seed_everything(seed=1234, workers=True)

In [ ]:
#no maxpool
mobile_structure1 = [{'conv':'real','in_channels':3, 'out_channels':64, 'stride':1, 'kernel_size':3, 'padding':1}, # shpae (-1,64,32,32)
             # first layer
             {'conv':'scaled_sign','in_channels':64, 'out_channels':64, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,64,32,32), Normal
             {'conv':'scaled_sign','in_channels':64, 'out_channels':64, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,64,32,32), Normal
             {'conv':'scaled_sign','in_channels':64, 'out_channels':64, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,64,32,32), Normal
             {'conv':'scaled_sign','in_channels':64, 'out_channels':64, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,64,32,32), Normal
             {'conv':'scaled_sign','in_channels':64, 'out_channels':64, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,64,32,32), Normal
             
             {'conv':'scaled_sign','in_channels':64, 'out_channels':128, 'stride':2, 'kernel_size':3, 'padding':1}, # output shape (-1,128,16,16), Reduction
             {'conv':'scaled_sign','in_channels':128, 'out_channels':128, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,128,16,16), Normal
             {'conv':'scaled_sign','in_channels':128, 'out_channels':128, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,128,16,16), Normal
             {'conv':'scaled_sign','in_channels':128, 'out_channels':128, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,128,16,16), Normal
             {'conv':'scaled_sign','in_channels':128, 'out_channels':128, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,128,16,16), Normal
                          
             {'conv':'scaled_sign','in_channels':128, 'out_channels':256, 'stride':2, 'kernel_size':3, 'padding':1}, # output shape (-1,256,8,8), Reduction
             {'conv':'scaled_sign','in_channels':256, 'out_channels':256, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,256,8,8), Normal
             {'conv':'scaled_sign','in_channels':256, 'out_channels':256, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,256,8,8), Normal
             {'conv':'scaled_sign','in_channels':256, 'out_channels':256, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,256,8,8), Normal
             {'conv':'scaled_sign','in_channels':256, 'out_channels':256, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,256,8,8), Normal
                          
             {'conv':'scaled_sign','in_channels':256, 'out_channels':512, 'stride':2, 'kernel_size':3, 'padding':1}, # output shape (-1,512,4,4), Reduction
             {'conv':'scaled_sign','in_channels':512, 'out_channels':512, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,512,4,4), Normal
             {'conv':'scaled_sign','in_channels':512, 'out_channels':512, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,512,4,4), Normal
             {'conv':'scaled_sign','in_channels':512, 'out_channels':512, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,512,4,4), Normal
             {'conv':'scaled_sign','in_channels':512, 'out_channels':512, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,512,4,4), Normal
             
             {'conv':'fc','in_channels':512, 'out_channels':10, 'stride':1, 'kernel_size':1, 'padding':0,'dropout':0.5}] # output shape (-1,10,1,1)



In [ ]:
'''
from model import ReactModel
from model import T_ReactModel

t_model = ReactModel(
    structure=mobile_structure1,
    adam_init_lr=0.01, 
    adam_weight_decay=0,
    adam_betas=(0.9, 0.999),
    lr_reduce_factor=0.1,
    lr_patience=50,
    limit_conv_weight=True,
    limit_bn_weight=True,
    teacher_model=None
              )

real_model = ReactModel.load_from_checkpoint('/content/drive/MyDrive/checkpoint/lightning_logs/LS_kldiv_0/version_0/checkpoints/epoch=318-val_loss=0.5379-val_acc=0.8726.ckpt')
t_model.load_state_dict(real_model.state_dict(), strict=False)

teacher_model=T_ReactModel(structure=mobile_structure1)

for i in t_model.state_dict():
  if i in teacher_model.state_dict():
    teacher_model.state_dict()[i]=t_model.state_dict()[i]

quadratic_model = ReactModel(
    structure=mobile_structure1,
    adam_init_lr=0.01, 
    adam_weight_decay=0,
    adam_betas=(0.9, 0.999),
    lr_reduce_factor=0.1,
    lr_patience=50,
    limit_conv_weight=True,
    limit_bn_weight=True,
    teacher_model=teacher_model
    )

print(quadratic_model.hparams)
print(quadratic_model)
'''

In [ ]:
from model import ReactModel

model = ReactModel(
    structure=mobile_structure1,
    adam_init_lr=0.001, 
    adam_weight_decay=0,
    adam_betas=(0.9, 0.999),
    lr_reduce_factor=0.1,
    lr_patience=50,
    limit_conv_weight=True,
    limit_bn_weight=True,
    teacher_model=None
    )

print(model.hparams)
print(model)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(filename='{epoch}-{val_loss:.4f}-{val_acc:.4f}', monitor='val_acc', mode='max')
trainer = Trainer(
    max_epochs=-1,
    gpus=AVAIL_GPUS,
    logger=TensorBoardLogger('lightning_logs/', name='no_teacher_binary_react', log_graph=True),
    # logger=TensorBoardLogger('lightning_logs/', name='Real', log_graph=True),
    callbacks=[LearningRateMonitor(logging_interval='step'), 
               EarlyStopping(monitor='val_acc', mode='max', patience=100),
               checkpoint_callback],
    deterministic=True,
#    gradient_clip_val = 0.5
)

trainer.fit(model, datamodule=cifar10_dm)

In [ ]:
trainer.test(model,cifar10_dm)